In [20]:
import analytics as an
from analytics.materials.ions import Cation, Anion
from analytics.materials.electrolytes import Electrolyte
from analytics.materials.solvents import Solvent
from analytics.experiment_modelling.cyclic_voltammetry import CyclicVoltammogram
import pandas as pd
import plotly.express as px

In [15]:
my_cation = Cation('Li+')
my_anion = Anion('Cl-')
my_solvent = Solvent('water')

my_electrolyte = Electrolyte(solvent=my_solvent, cation=my_cation, anion=my_anion, concentrations={my_cation: 0.1, my_anion: 0.1})


In [16]:

my_cv = CyclicVoltammogram.from_benelogic("../test_trajectories/cyclic_voltammetry/benelogic1.txt", electrolyte=my_electrolyte)

In [19]:
my_cv.data

,potential,current,cycle,time,redox
405,0.188631,0.000131,0,40.028799,oxidation
406,0.193625,0.000131,0,40.128799,oxidation
407,0.198630,0.000131,0,40.228799,oxidation
408,0.203632,0.000131,0,40.328799,oxidation
409,0.208632,0.000131,0,40.428799,oxidation
...,...,...,...,...,...
2011,0.145841,0.000134,4,199.171795,oxidation
2012,0.150837,0.000134,4,199.271795,oxidation
2013,0.155851,0.000134,4,199.371795,oxidation
2014,0.159644,0.000133,4,199.447795,oxidation


In [42]:

def add_endpoints(df):
    first_redox = df['redox'].iloc[0]
    last_redox = df['redox'].iloc[-1]
    last_index = df.query('redox == @first_redox').index[-1]
    first_chunk = df.query('redox == @first_redox')
    second_chunk = df.query('redox == @first_redox and index == @last_index').assign(redox = last_redox)
    third_chunk = df.query('redox == @last_redox') 
    return pd.concat([first_chunk, second_chunk, third_chunk])

data = (my_cv
        .data
        .query('cycle == 1')
        .groupby(['cycle'], group_keys=False)
        .apply(lambda df: df.pipe(add_endpoints))
        .assign(tag = lambda x: x['cycle'].astype(str) + x['redox'].astype(str))
        )

px.line(data, x='time', y='current', color='tag', title='Cyclic Voltammogram', template='plotly_dark', width=1200, height=600, markers=True).show()